# Cart Pendulum: Hamiltonian Dynamics

In [ ]:
from ipywidgets import interact
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from numerical_analysis import *
from matplotlib import animation, rc
from IPython.display import HTML

import sympy  as sp
import numpy as np
import control as ct
import control.matlab as ct_matlab

import slycot


# Deriving the equations

### Theory

##### Defining the variables
In Hamiltonian formulation for this system, the system states are \
$x_1$: Position of the cart along the horizontal (x-axis) \
$p_1$: The conjugate momentum of $x_1$ \
$x_2$: The angle measured along the negative y-axis \
$p_2$: The conjugate momentum of $x_2$ 

Note that the conjugate momentum need not be the momentum\angular momentum, such as $p = mv$ or $J = mr^2\omega$, we use in classical mechanics. It is calculated when we use the Legendre transformation to convert the Lagrangagian to Hamiltonian and is defined by the following equation

$
\begin{equation}
    p_i = \frac{\partial L}{\partial x_1}
\end{equation}
$

The Lagrangian $L$ of this system is defined as 
$
\begin{equation}
    L = ke - pe
\end{equation}
$
where $ke$ is the kinetic energy and $pe$ is the potential energy. Now we substitute $L$ in equation (1) and solve for the velocities as a function of the positions and conjugate momenta.

Next we calculate the Hamiltonian using the Legendre transformation
$
\begin{equation}
    H = x_1 p_1 + x_2 p_2 - L \\
\end{equation}
$

Finally we get the dynamics of this system using the Hamiltons equations, modified to include a control signal
$
\begin{equation}
    \begin{aligned}
        \dot{x_1} &=   -\frac{\partial H}{\partial p_1},  \space \space  \dot{p_1} =   \frac{\partial H}{\partial x_1} - u\\
        \dot{x_2} &=   -\frac{\partial H}{\partial p_2},  \space \space  \dot{p_2} =   \frac{\partial H}{\partial x_2}  
    \end{aligned}
\end{equation}
$

Here $u$ is the control signal

For more information on Hamiltonian dynamics, I recommend visiting this class taught by a master \
https://www.youtube.com/watch?v=GOkZs2RZMQY&list=PL5E4E56893588CBA8&index=10   

### Setting up positions and velocities

In [ ]:
# The function for differentiation is cumbersome
# Defining lambdas for brevity
ddx = lambda f,x : sp.diff(f,x)
ddt = lambda f   : sp.diff(f,t)

# System parameters symbols
m1, m2, l, g = sp.symbols('m1, m2, l, g')

# Dictionary to store their numerical values
param_vals = {
    "m1" : 1,
    "m2" : 1,
    "g"  : -10,
    "l"  : 1
}

# Time and control input symbols
t, u =  sp.symbols('t, u')

# System state function symbols
# They are functions of time
# Position
x1 = sp.Function('x1')
x2 = sp.Function('x2')
# Conjugate momenta
p1 = sp.Function('p1')
p2 = sp.Function('p2')

# Velocities variables
v1 = ddt(x1(t))
v2 = ddt(x2(t))

# Defining the position in the x-y plane
cart_x = x1(t)
cart_y = 0
cart_pos = [cart_x, cart_y]

pend_x =  l*sp.sin(x2(t)) + x1(t)
pend_y = -l*sp.cos(x2(t))
pendulum_pos = [pend_x, pend_y]

# Calculating the velocities
cart_vel = [ddt(x) for x in cart_pos]
pend_vel = [ddt(x) for x in pendulum_pos]

# Calculating the sum of velocity squared
cart_vel_squared = sum(v**2 for v in cart_vel)
pend_vel_squared = sum(v**2 for v in pend_vel)

### Calculating energy terms

In [ ]:
# Kinetic energy
ke = m1*cart_vel_squared/2 + m2*pend_vel_squared/2

# Potential energy
pe = -m2*g*pend_y

# Lagrangian: 
L  = ke - pe
# We are using the Langragian to express velocities as function 
# of conjugate momenta and to calculate the Hamiltonian using 
# a legendre transformation

# Equations connecting conjugate momenta and velocities
conjugate_momentum_eqs = [
    sp.simplify(p1(t) - ddx(L, v1)),
    sp.simplify(p2(t) - ddx(L, v2))
]

# Velcoties as a function of ps
vels_sol = sp.solve(conjugate_momentum_eqs, [v1, v2])
vels_sol = {v:vels_sol[v].simplify() for v in [v1, v2]}

# Legendre transformation: Lagrangian to Hamiltonian
H = p1(t)*v1 + p2(t)*v2 - L
H = H.subs(vels_sol).simplify()



#### Dynamics equations

In [ ]:
# Hamiltons equations 
x1d_exp =  ddx(H, p1(t)).simplify()
p1d_exp = -ddx(H, x1(t)).simplify() - u
x2d_exp =  ddx(H, p2(t)).simplify()
p2d_exp = -ddx(H, x2(t)).simplify()

# Expressions to functions
to_eq    = lambda eq:sp.lambdify((x1(t), p1(t), x2(t), p2(t), t, u, m1, m2, l, g), eq)
DE_eqs   = [x1d_exp, p1d_exp, x2d_exp, p2d_exp]
DE_funcs = [to_eq(eq) for eq in DE_eqs]

### System dynamics function

In [ ]:
def get_PendulumCartRates_Hamiltonian(X, t, uf, m1, m2, l, g):
    X_  = X
    u   = uf(X_)
    x1, p1, x2, p2 = X_

    return [f(x1, p1, x2, p2, t, u, m1, m2, l, g) for f in DE_funcs]

# Unforced dynamics

#### Animation plotting 

In [ ]:
def animte_CartPend(result, ts, Xs):
    """
    Accepts cart state information and returns an animation
    """
    # Extracting the symbols
    x1, x2 = Xs

    # Setting axes range
    x_range = [-3,   3]
    y_range = [-1.1, 1.1]
    delta   = lambda ls: ls[1] - ls[0]
    aspect  = (delta(x_range)) / (delta(y_range))

    # setting matplotlib parameters
    scale = 3
    plt.rcParams['figure.figsize'] = [aspect * scale, scale]
    plt.rcParams.update({'font.size': 18})
    plt.rcParams['animation.html'] = 'jshtml'


    # Selecting the animation range
    res_plot = result[::50]
    ts_plot  = ts[::50]

    # Setting up the plot
    fig, ax     = plt.subplots()
    pend_plot,  = plt.plot([],[], 'o-', linewidth=2, ms=10, markerfacecolor='r')
    cart_plot,  = plt.plot([],[], 'ks', ms=10) 

    print(type(pend_plot))

    def init():
        ax.set_xlim(x_range[0], x_range[1])
        ax.set_ylim(y_range[0], y_range[1])
        ax.set_aspect('equal')

    def animate(iter):
        x_cart, _, th_pend, _ = res_plot[iter]
        sub_obs               = {x1: x_cart, x2: th_pend, l: param_vals["l"]}
        x_pend, y_pend        = [sp.N(i.subs(sub_obs)) for i in pendulum_pos]

        pend_plot.set_data([x_cart, x_pend],[0, y_pend])
        cart_plot.set_data([x_cart]        ,[0])

    return animation.FuncAnimation(fig, animate, init_func=init, frames=len(res_plot), interval=50, blit=False, repeat=False)

#### Simlation

In [ ]:
# Simulation parameters
dt = 0.001
t0 = 0
tn = 10
ts = np.arange(t0, tn, dt)
t_span = [0,dt]

# initial condition
x0  = [0,0,0,2]

# We are not controlling. A trial control law
control_no = lambda x:0

# Integrations
out = odeint(get_PendulumCartRates_Hamiltonian, x0, ts, 
    args=(control_no, param_vals["m1"], param_vals["m2"], param_vals["l"], param_vals["g"])
)

#### Animation

In [ ]:
anim = animte_CartPend(out, ts, (x1(t), x2(t)))
HTML(anim.to_jshtml())

# Control

### Deriving the Linearised models

In [ ]:
Xs  = [x1(t),   p1(t),      x2(t),   p2(t)]
Xds = [x1d_exp, p1d_exp, x2d_exp, p2d_exp]


#  the stationary point Substitution. 
# We are linearising about the vertical, mass up position
subs_stationary = {
    x2(t) : sp.pi,
    p1(t) : 0,
    p2(t) : 0
}

A_sp = sp.Matrix([ [  ddx(xd,x).simplify().subs(subs_stationary) for x in Xs]  for xd in Xds])
B_sp = sp.Matrix([ [  ddx(xd,u).simplify().subs(subs_stationary)]              for xd in Xds])

### Using LQR to calculate the gain

In [ ]:
# The Parameters for substitution to get numerical values
param_subs = {
    m1 : param_vals["m1"],
    m2 : param_vals["m2"],
    g  : param_vals["g"],
    l  : param_vals["l"] 
}

# Matrices with numerical values
A_np  = np.array(A_sp.subs(param_subs))
B_np  = np.array(B_sp.subs(param_subs))
Q     = np.eye(4)
R     = 0.001

# Use the LQR algorithm 
K_np  = np.array(ct_matlab.lqr(A_np, B_np, Q, R)[0])

### Simulating balancing the pendulum

In [ ]:

# Simulation parameters
dt = 0.001
t0 = 0.0
tn = 10.0
ts = np.arange(t0, tn, dt)
t_step = [0.0, dt]

# Initial condition
x0  = np.array([0, 0, np.pi-0.8, 0])

# Set point
xsp = np.array([0, 0, np.pi, 0])

# Controller function
control_law = lambda x: -(K_np @(x-xsp))[0]

# Integrate
res_ctrl = odeint(
    get_PendulumCartRates_Hamiltonian, x0, ts, 
    args=(control_law, param_vals["m1"], param_vals["m2"], param_vals["l"], param_vals["g"])
)


### Animation of the result

In [ ]:
anim = animte_CartPend(res_ctrl, ts, (x1(t), x2(t)))
HTML(anim.to_jshtml())